In [1]:
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles
import pandas as pd
from pyspark.sql.types import IntegerType

### Question 1: 

**Install Spark and PySpark** 

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

In [2]:
spark = (SparkSession.builder
         .master("local[4]")
         .appName("test")
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 08:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.0'

### Question 2: 

**FHV October 2019**

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 1MB
- 6MB
- 25MB
- 87MB

In [4]:
file_name = 'fhv_tripdata_2019-10.csv.gz'
file_url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{file_name}'
spark.sparkContext.addFile(file_url)

### schema inference with pandas trick

In [5]:
# Read the first 100 rows using Pandas
first_100_rows = pd.read_csv(file_url, compression='gzip', nrows=100, parse_dates=['pickup_datetime', 'dropOff_datetime'])
first_100_rows_sp = spark.createDataFrame(first_100_rows)
first_100_rows_sp = (first_100_rows_sp
                     .withColumn("SR_Flag", first_100_rows_sp["SR_Flag"].cast(IntegerType()))
                     .withColumn("DOLocationID", first_100_rows_sp["DOLocationID"].cast(IntegerType()))
                     .withColumn("PULocationID", first_100_rows_sp["PULocationID"].cast(IntegerType())))
first_100_rows_sp.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropOff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', IntegerType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [6]:
df = spark.read.csv(SparkFiles.get(file_name), header=True, schema=first_100_rows_sp.schema)

### Write the obtained csv to partitioned parquet

In [7]:
df.repartition(6).write.parquet('fhv/2019/10/', mode='overwrite')

In [8]:
!ls -lh fhv/2019/10/

total 83896
-rw-r--r--@ 1 alexlitvinov  staff     0B Mar  6 08:16 _SUCCESS
-rw-r--r--@ 1 alexlitvinov  staff   6.4M Mar  6 08:16 part-00000-3fe45406-900f-4d51-903d-2e2f4891ca8c-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   6.3M Mar  6 08:16 part-00001-3fe45406-900f-4d51-903d-2e2f4891ca8c-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   6.4M Mar  6 08:16 part-00002-3fe45406-900f-4d51-903d-2e2f4891ca8c-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   6.4M Mar  6 08:16 part-00003-3fe45406-900f-4d51-903d-2e2f4891ca8c-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   6.4M Mar  6 08:16 part-00004-3fe45406-900f-4d51-903d-2e2f4891ca8c-c000.snappy.parquet
-rw-r--r--@ 1 alexlitvinov  staff   6.4M Mar  6 08:16 part-00005-3fe45406-900f-4d51-903d-2e2f4891ca8c-c000.snappy.parquet


### Question 3: 

**Count records** 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

- 108,164
- 12,856
- 452,470
- 62,610

> [!IMPORTANT]
> Be aware of columns order when defining schema


In [9]:
df.createOrReplaceTempView('fhv')

In [10]:
spark.sql("""
select count(1)
from fhv
where DATE(pickup_datetime) = '2019-10-15'
""").show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



### Question 4: 

**Longest trip for each day** 

What is the length of the longest trip in the dataset in hours?

- 631,152.50 Hours
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours

In [11]:
spark.sql("""
SELECT max((unix_timestamp(dropOff_datetime) - unix_timestamp(pickup_datetime))/3600) as max_diff_in_hours
FROM fhv
ORDER BY 1 desc
""").show()

+-----------------+
|max_diff_in_hours|
+-----------------+
|         631152.5|
+-----------------+




### Question 5: 

**User Interface**

Spark’s User Interface which shows the application's dashboard runs on which local port?

- 80
- 443
- 4040
- 8080



In [13]:
spark_ui_port = spark.sparkContext.uiWebUrl.split(":")[-1]

print("Spark UI port:", spark_ui_port)


Spark UI port: 4040


### Question 6: 

**Least frequent pickup location zone**

Load the zone lookup data into a temp view in Spark</br>
[Zone Data](https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv)

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?</br>

- East Chelsea
- Jamaica Bay
- Union Sq
- Crown Heights North

In [14]:
zones_url = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'
spark.sparkContext.addFile(zones_url)

In [15]:
df_zones = spark.read.csv(SparkFiles.get('taxi_zone_lookup.csv'), header=True, inferSchema=True)

In [18]:
df_zones.createOrReplaceTempView('zones')

In [22]:
spark.sql("""
select zones.Zone, count(1) desc
from fhv
join zones on zones.LocationID = fhv.PULocationID
group by zones.Zone
order by 2 asc
""").show()

+--------------------+----+
|                Zone|desc|
+--------------------+----+
|         Jamaica Bay|   1|
|Governor's Island...|   2|
| Green-Wood Cemetery|   5|
|       Broad Channel|   8|
|     Highbridge Park|  14|
|        Battery Park|  15|
|Saint Michaels Ce...|  23|
|Breezy Point/Fort...|  25|
|Marine Park/Floyd...|  26|
|        Astoria Park|  29|
|    Inwood Hill Park|  39|
|       Willets Point|  47|
|Forest Park/Highl...|  53|
|  Brooklyn Navy Yard|  57|
|        Crotona Park|  62|
|        Country Club|  77|
|     Freshkills Park|  89|
|       Prospect Park|  98|
|     Columbia Street| 105|
|  South Williamsburg| 110|
+--------------------+----+
only showing top 20 rows

